In [5]:
# poröse layer und air-layer

import numpy as np
from numpy import cos, sin, pi, sqrt, abs, dot
import plotly.graph_objs as go

In [6]:
# JAC-Modell
# Porosität des porösen Materials (phi Φ) aus Tabelle
# Dichte der Flüssigkeit (normalerweise Luft) (ρ)
# Viskosität des Fluids (η)
# Strömungswiderstand des porösen Materials (σ)
# Frequenz der Schallwelle (f)
# Thermische Parameter eines porösen Materials (Pr)


def JAC(f, phi, sigma, alpha_infi, P0, density_0, viskosity):

# External variables
    gamma = 1.4 #gamma is ratio of the specific heat capacities (Approximately 1.4),
    K0 = gamma * P0 # P0 is atmospheric pressure 101,320Nm^-2;
    omega = 2*pi*f
    kappa = 0.0241 # thermal conductivity of air
    cp = 1.01 # specific heat capacity of air
    delta_v = np.sqrt(2*viskosity/(density_0*omega))  # delta_v and delta_h are the sizes of the viscous and thermal boundary layers
    delta_h = np.sqrt(2*kappa/(density_0*omega*cp))
    Pr = (delta_v/delta_h)**2 # Prandtl number

    s = 1 # constant, for simpler formulations s=1
    viskosity_L = 1/s*sqrt(8*viskosity*alpha_infi/(phi*sigma))
    thermal_L= 2*viskosity_L #simpler formulations

    G1 = sigma * phi / (alpha_infi * density_0 * omega)
    G2 = 4*((alpha_infi)**2) * density_0 * viskosity * omega / ((sigma*phi*viskosity_L)**2)

    G1_dot = 8*viskosity / (density_0 * Pr * ((thermal_L)**2) * omega)
    G2_dot = density_0 * Pr * ((thermal_L)**2) * omega / (16*viskosity)
    
    density_p = density_0 * alpha_infi * (1- 1j*G1*sqrt(1+1j*G2)) / phi # effective density of the porous material

    K_p = K0*phi**(-1) / (gamma - (gamma-1)*((1- 1j*G1_dot*sqrt(1+ 1j*G2_dot))**-1)) # effective or dynamic bulk modulus of the air in the material
    
    kp = omega * sqrt(density_p/K_p) # propagation wavenumber

    Zp = sqrt(density_p*K_p) # characteristic impedance

    return Zp, kp



In [7]:
def tmm(
        theta: float,
        k0: float,
        k1: float,
        L1: float,
        Z1: float,
        k2: float,
        L2: float,
        Z2: float
    ) -> None:

    ##  k_x, k_z bei Eingallswinkel theta
    k_0x = k0*sin(theta)
    k_1x = k_0x
    k_2x = k2*sin(theta)

    k_1z = sqrt(k1**2 - k_1x**2)
    k_2z = sqrt(k2**2 - k_2x**2)

    # T1, T2, T3 definieren

    T1 = np.array([[cos(k_1z*L1), 1j*Z1*(k1/k_1z)*sin(k_1z*L1)],
                [(1j/Z1)*(k_1z/k1)*sin(k_1z*L1), cos(k_1z*L1)]])

    T2 = np.array([[cos(k_2z*L2), 1j*Z2*(k2/k_2z)*sin(k_2z*L2)],
                [(1j/Z2)*(k_2z/k2)*sin(k_2z*L2), cos(k_2z*L2)]])

    # Rigid Backed
    T3 = np.array([[1, 0],  
                [0, 1]])

    # T_total berechnen
    T_total = dot(T1, T2)

    return T_total

In [8]:
## Schräge Inzidenz (Oblique Incidence)
## Paremeter 
air_speed = 344
air_density = 1.213  # dichte von Luft-Schicht
viskosity = 1.839*10**(-5)
P0 = 101325 # Pressure Pa

# Parameters
phi = 0.98
# material variables
# • For fibrous material with thermoplastic bi-component fibers binding or in the
# absence of thermoset resin binding α∞ = 1.
# • For foam material with open membranes (high reticulation) after typical free
# expansion 1.1 < α∞ < 1.4.
# • For foam material with semi-closed membranes (low reticulation) after typical
# injection molding expansion or relatively closed cell free expansion 1.5 < α∞ <
# 2.5.
alpha_infi = 1.4 # here a checkbox for choice
sigmas = [2600, 2600]

# gamma = 1.4
# viskosität_L = 85*10**(-6)
# thermisch_L = viskosität_L*2
# Pr = 0.71
# viskosität = 1.839*10**(-5)

# Frequenz Bereich
L1 = float(input('poroese Schicht Dicke(mm):'))
L1 = L1/1000
L2 = float(input('Luft Schicht Dicke(mm):'))
L2 = L2/1000
angle = float(input('Einfallswinkel(Grad):'))
theta = np.pi*angle/180 # theta nur rad sein können
frequency_min = 10
frequency_max = float(input('max. frequenz:'))
frequency_range = np.arange(frequency_min, frequency_max)


len = int(frequency_max - frequency_min)
alphas = np.zeros((2,len)) # Array speichert Absorptionsgrad
# alphas = np.array([]) # Array speichert Absorptionsgrad

# Impedanz
Z0 = air_speed * air_density
#Z2 = luft_dichte * luft_c  # Luft-Schicht

for i in range(2):
    sigma = sigmas[i]
    j =0
    for f in frequency_range:
        Z1, k1 = JAC(f, phi, sigma, alpha_infi, P0, air_density, viskosity)
        Z2 = air_density * air_speed

        k0 = 2*np.pi*f / (air_speed)
        k2 = k0

        T_total = tmm(theta,k0,k1,L1,Z1,k2,L2,Z2)
        
        # Absorptionsgrad berechnen
        R = (T_total[0, 0]*cos(theta) - Z0*T_total[1, 0]) / (T_total[0, 0]*cos(theta) + Z0*T_total[1, 0]) # Reflexionskoeffizient
        alpha = 1 - (abs(R)**2)
        alphas[i,j] = alpha
        j = j+1
   

# Ersetzen der Werte durch 0
alphas[(alphas < 0.000001) & (alphas > 0)] = 0

# Diagramm, Grad über Frequenz
fig = go.Figure()
fig.add_trace(go.Scatter(x=frequency_range, y=alphas[0,:]))
fig.update_xaxes(type="log")
fig.show()

In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=frequency_range, y=alphas[1,:]))
fig.show()